## Exploring the banking dataset using AutoLabel

#### Setup the API Keys for providers that you want to use

In [1]:
# %env OPENAI_API_KEY=***REMOVED***
%env OPENAI_API_KEY=***REMOVED***
%env ANTHROPIC_API_KEY=***REMOVED***

env: OPENAI_API_KEY=***REMOVED***
env: ANTHROPIC_API_KEY=***REMOVED***


### Get the correct data

First lets make sure that we download the correct data. Currently data has been hosted on s3 but we will upload it to huggingface data as well in the future.

In [ ]:
!aws s3 cp s3://refuel-benchmarking data/ --recursive

### Run the labeler after passing in my own seed examples

#### Create the config
This config file has all the possible labels for the banking dataset. The model needs to choose one label from the label list provided. In input schema, we define the input columns that will be used by the oracle and the output column defines the label column, that is, the column that will be used as ground truth and will be tried to be generated by our library

In [10]:
zero_shot_config = {
    "task_name": "ProductCatalogEntityMatch",
    "task_type": "entity_matching",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo",
        "params": {}
    },
    "prompt": {
        "task_guidelines": "You are an expert at identifying duplicate products from online product catalogs.\nYour job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:\n{labels}",
        "labels": ["duplicate", "not duplicate"],
        "example_template": "Title of entity1: {Title_entity1}; category of entity1: {Category_entity1}; brand of entity1: {Brand_entity1}; model number of entity1: {ModelNo_entity1}; price of entity1: {Price_entity1}\nTitle of entity2: {Title_entity2}; category of entity2: {Category_entity2}; brand of entity2: {Brand_entity2}; model number of entity2: {ModelNo_entity2}; price of entity2: {Price_entity2}\nDuplicate or not: {label}"
    }
}


## Run the model

First dry run the model using the above specification and get an idea of the cost required to run the model 

Running it on just 100 examples to get an idea for the notebook, adjust the max_items as required and dont pass it if you need to run the oracle on the whole dataset.

In [2]:
from autolabel import LabelingAgent

In [11]:
agent = LabelingAgent(config=zero_shot_config)

In [12]:

agent.plan('../data/walmart_amazon_test_multicol.csv', max_items=10)


Output()

Total Estimated Cost: $0.024
Number of examples to label: 10
Average cost per example: $0.00239


A prompt example:

You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the correct option"


Now I want you to label the following example:
Title of entity1: sony 16gb class 4 sd memory card; category of entity1: usb drives; brand of entity1: sony; model number of entity1: sf16n4/tqp; price of entity1: 0.0
Title of entity2: pny 4gb class 4 navy sd card; category of entity2: car audio video; brand of entity2: pny; model number of entity2: p-sdhc4g4-ef / navy; price of entity2: 11.18
Duplicate or not: 


Now, actually run the model and generate the list of labels for the banking dataset. You will get the computed metrics at the end of the run.

In [14]:
labels, df, metrics_list = agent.run(
    '../data/walmart_amazon_test_multicol.csv', max_items=500)


Output()

2023-05-31 16:13:38 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89197 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:13:38 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89197 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:13:40 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88921 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:13:40 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88921 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:13:42 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89256 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:13:42 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89256 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:13:44 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88820 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:13:44 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88820 / min. Contact us through our help center at help.openai.com if you continue to have issues..


...

2023-05-31 16:17:36 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89337 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:17:36 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89337 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:17:37 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88995 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:17:37 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88995 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:17:40 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89006 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:17:40 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89006 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:17:41 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89265 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:17:41 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89265 / min. Contact us through our help center at help.openai.com if you continue to have issues..


Metric: Actual Cost: 0.15630399999999997
Metric: support: [(500, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.916, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


## Change the prompt
Analyze the dataframe generated above and make changes to the prompt in order to get better results

In [9]:
config_fixed_few_shot = {
    "task_name": "ProductCatalogEntityMatch",
    "task_type": "entity_matching",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo",
        "params": {}
    },
    "prompt": {
        "task_guidelines": "You are an expert at identifying duplicate products from online product catalogs.\nYour job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:\n{labels}",
        "labels": ["duplicate", "not duplicate"],
        "example_template": "Title of entity1: {Title_entity1}; category of entity1: {Category_entity1}; brand of entity1: {Brand_entity1}; model number of entity1: {ModelNo_entity1}; price of entity1: {Price_entity1}\nTitle of entity2: {Title_entity2}; category of entity2: {Category_entity2}; brand of entity2: {Brand_entity2}; model number of entity2: {ModelNo_entity2}; price of entity2: {Price_entity2}\nDuplicate or not: {label}",
        "few_shot_examples": [
            {
                "Title_entity1": "lexmark extra high yield return pgm print cartridge - magenta",
                "Category_entity1": "printers",
                "Brand_entity1": "lexmark",
                "ModelNo_entity1": "c782u1mg",
                "Price_entity1": "214.88",
                "Title_entity2": "lexmark 18c1428 return program print cartridge black",
                "Category_entity2": "inkjet printer ink",
                "Brand_entity2": "lexmark",
                "ModelNo_entity2": "18c1428",
                "Price_entity2": "19.97",
                "label": "not duplicate"
            },
            {
                "Title_entity1": "edge tech proshot 4gb sdhc class 6 memory card",
                "Category_entity1": "usb drives",
                "Brand_entity1": "edge tech",
                "ModelNo_entity1": "pe209780",
                "Price_entity1": "10.88",
                "Title_entity2": "4gb edge proshot sdhc memory card class6",
                "Category_entity2": "computers accessories",
                "Brand_entity2": "edge",
                "ModelNo_entity2": "nan",
                "Price_entity2": "17.83",
                "label": "duplicate"
            }
        ],
        "few_shot_selection": "fixed",
        "few_shot_num": 2,
    }
}


In [16]:
agent = LabelingAgent(config_fixed_few_shot)
agent.plan('../data/walmart_amazon_test_multicol.csv', max_items = 100)

Output()

Total Estimated Cost: $0.288
Number of examples to label: 100
Average cost per example: $0.00288


A prompt example:

You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Title of entity1: lexmark extra high yield return pgm print cartridge - magenta; category of entity1: printers; brand of entity1: lexmark; model number of entity1: c782u1mg; price of entity1: 214.88
Title of entity2: lexmark 18c1428 return program print cartridge black; category of entity2: inkjet printer ink; brand of entity2: lexmark; model number of entity2: 18c1428; price of entity2: 19.97
Duplicate or not: not duplicate

Title of entity1: edge tech proshot 4gb sdhc class 6 memory card; category of enti

In [18]:
labels, df, metrics_list = agent.run(
    '../data/walmart_amazon_test_multicol.csv', max_items=500)


Output()

2023-05-31 16:21:58 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89219 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:21:58 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89219 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:21:59 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89211 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:21:59 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89211 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:22:01 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89297 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:22:01 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89297 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:22:03 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88669 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:22:03 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88669 / min. Contact us through our help center at help.openai.com if you continue to have issues..


...

2023-05-31 16:26:23 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88649 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:26:23 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88649 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:26:25 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88668 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:26:25 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88668 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:26:27 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88899 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:26:27 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88899 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 16:26:28 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88977 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 16:26:28 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88977 / min. Contact us through our help center at help.openai.com if you continue to have issues..


Metric: Actual Cost: 0.35156799999999994
Metric: support: [(500, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.962, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


In [19]:
from collections import Counter
Counter(labels.tolist())

Counter({'not duplicate': 455, 'duplicate': 45})

## Example selector
Let's try to use an example selector. Let's use the big seed example set so that we can choose from a bigger set of examples

In [3]:
config_similarity_few_shot = {
    "task_name": "ProductCatalogEntityMatch",
    "task_type": "entity_matching",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo",
        "params": {}
    },
    "prompt": {
        "task_guidelines": "You are an expert at identifying duplicate products from online product catalogs.\nYour job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:\n{labels}",
        "labels": ["duplicate", "not duplicate"],
        "example_template": "Title of entity1: {Title_entity1}; category of entity1: {Category_entity1}; brand of entity1: {Brand_entity1}; model number of entity1: {ModelNo_entity1}; price of entity1: {Price_entity1}\nTitle of entity2: {Title_entity2}; category of entity2: {Category_entity2}; brand of entity2: {Brand_entity2}; model number of entity2: {ModelNo_entity2}; price of entity2: {Price_entity2}\nDuplicate or not: {label}",
        "few_shot_examples": "../data/walmart_amazon_seed_multicol.csv",
        "few_shot_selection": "semantic_similarity",
        "few_shot_num": 3,
    }
}


In [4]:
agent = LabelingAgent(config_similarity_few_shot)


In [5]:
agent.plan('../data/walmart_amazon_test_multicol.csv', max_items=100)


Output()

Total Estimated Cost: $0.318
Number of examples to label: 100
Average cost per example: $0.00318


A prompt example:

You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Title of entity1: wintec filemate 4gb compactflash memory card; category of entity1: usb drives; brand of entity1: wintec; model number of entity1: 3fmcf4gb-r; price of entity1: 15.98
Title of entity2: sandisk 4gb ultra ii compactflash memory card 15mb s; category of entity2: memory cards; brand of entity2: sandisk; model number of entity2: sdcfh-4096; price of entity2: 17.96
Duplicate or not: not duplicate

Title of entity1: dane-elec 8gb high-speed compact flash memory card; category of entity1: usb drive

In [6]:
labels, df, metrics_list = agent.run(
    '../data/walmart_amazon_test_multicol.csv', max_items=500)


Output()

2023-05-31 18:00:06 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89386 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:00:06 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89386 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 18:00:07 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88903 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:00:07 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88903 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 18:00:15 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89286 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:00:15 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89286 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 18:00:16 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88662 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:00:16 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88662 / min. Contact us through our help center at help.openai.com if you continue to have issues..


...

2023-05-31 18:05:51 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89397 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:05:51 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89397 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 18:05:53 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88823 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:05:53 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88823 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 18:05:56 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88977 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:05:56 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88977 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-31 18:05:57 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89390 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-31 18:05:57 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89390 / min. Contact us through our help center at help.openai.com if you continue to have issues..


Metric: Actual Cost: 0.5878560000000002
Metric: support: [(500, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.95, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


In [10]:
## Davinci

from copy import deepcopy

config_v4 = deepcopy(config_fixed_few_shot)

config_v4['model'] = {'provider': 'openai', 'name': 'text-davinci-003'}


In [11]:
agent = LabelingAgent(config_v4)

In [12]:
agent.plan('../data/walmart_amazon_test_multicol.csv', max_items=500)


Output()

Total Estimated Cost: $14.513
Number of examples to label: 500
Average cost per example: $0.02903


A prompt example:

You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Title of entity1: lexmark extra high yield return pgm print cartridge - magenta; category of entity1: printers; brand of entity1: lexmark; model number of entity1: c782u1mg; price of entity1: 214.88
Title of entity2: lexmark 18c1428 return program print cartridge black; category of entity2: inkjet printer ink; brand of entity2: lexmark; model number of entity2: 18c1428; price of entity2: 19.97
Duplicate or not: not duplicate

Title of entity1: edge tech proshot 4gb sdhc class 6 memory card; category of ent

In [13]:
labels, df, metrics_list = agent.run(
    '../data/walmart_amazon_test_multicol.csv', max_items=500)


Output()

Metric: Actual Cost: 4.518239999999999
Metric: support: [(500, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.738, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


In [14]:

config_v5 = deepcopy(config_fixed_few_shot)

config_v5['model'] = {'provider': 'openai', 'name': 'gpt-4'}


In [15]:
agent = LabelingAgent(config_v5)

In [16]:
agent.plan('../data/walmart_amazon_test_multicol.csv', max_items=10)

Output()

Total Estimated Cost: $0.732
Number of examples to label: 10
Average cost per example: $0.07321


A prompt example:

You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Title of entity1: lexmark extra high yield return pgm print cartridge - magenta; category of entity1: printers; brand of entity1: lexmark; model number of entity1: c782u1mg; price of entity1: 214.88
Title of entity2: lexmark 18c1428 return program print cartridge black; category of entity2: inkjet printer ink; brand of entity2: lexmark; model number of entity2: 18c1428; price of entity2: 19.97
Duplicate or not: not duplicate

Title of entity1: edge tech proshot 4gb sdhc class 6 memory card; category of entit

In [20]:
labels, df, metrics_list = agent.run(
    '../data/walmart_amazon_test_multicol.csv', max_items=100)


2023-05-31 22:06:33.237 | INFO     | autolabel.labeler:run:138 - Task run already exists.
There is an existing task with following details: id='634053474' created_at=datetime.datetime(2023, 5, 31, 21, 43, 40, 157424) task_id='73897c18e2e87ceb58e1d3d203aefb67' dataset_id='4556ed271b98bc599bd9234150639b84' current_index=45 output_file='../data/walmart_amazon_test_multicol_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None
Evaluating the existing task...
Metric: support: [(50, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.96, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
50 examples have been labeled so far.
Last annotated example - Prompt: You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the cor

Output()

Resuming the task...


2023-05-31 22:08:34 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d5a1dd557b9df27b06fc035a30cb2e4c in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-31 22:08:34 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d5a1dd557b9df27b06fc035a30cb2e4c in your message.).


KeyboardInterrupt: 

In [21]:

config_v6 = deepcopy(config_fixed_few_shot)

config_v6['model'] = {'provider': 'anthropic', 'name': 'claude-v1'}


In [22]:
agent = LabelingAgent(config_v6)
agent.llm


In [23]:
agent.plan('../data/walmart_amazon_test_multicol.csv' , max_items=10)

Output()

Total Estimated Cost: $0.375
Number of examples to label: 10
Average cost per example: $0.03753


A prompt example:

You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Title of entity1: lexmark extra high yield return pgm print cartridge - magenta; category of entity1: printers; brand of entity1: lexmark; model number of entity1: c782u1mg; price of entity1: 214.88
Title of entity2: lexmark 18c1428 return program print cartridge black; category of entity2: inkjet printer ink; brand of entity2: lexmark; model number of entity2: 18c1428; price of entity2: 19.97
Duplicate or not: not duplicate

Title of entity1: edge tech proshot 4gb sdhc class 6 memory card; category of entit

In [24]:
labels, df, metrics_list = agent.run(
    '../data/walmart_amazon_test_multicol.csv', max_items=100)


Output()

Metric: Actual Cost: 0.48995679999999997
Metric: support: [(100, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.61, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


In [ ]:
from collections import Counter

Counter(labels.tolist())

In [ ]:

config_refuel = deepcopy(config_similarity_few_shot)

config_refuel['model'] = {'provider': 'refuel', 'name': 'flan-t5-xxl'}


In [ ]:
agent = LabelingAgent(config_refuel)
agent.llm


In [ ]:
agent.plan('../data/banking_test.csv', max_items=10)

In [ ]:
labels, df, metrics_list = agent.run('../data/banking_test.csv', max_items=10)


## Confidence estimation
Let's try to see if the model is able to do well on confidence estimation, if the model knows what it doesn't know we might be able to trade off completion rate for accuracy and get a higher accuracy even though we may be labeling less amount of data.

In [ ]:
task_config = {
    "project_name": "BankingClassification",
    "task_type": "classification",
    "prefix_prompt": "You are an expert at understanding twitter complaints.",
    "example_selector": {
        "strategy": "semantic_similarity",
        "num_examples": 4
    },
    "compute_confidence": "True"
}

In [ ]:
o = LabelingAgent(task_config, llm_config)
o.plan('data/banking_test.csv', dataset_config, max_items = 100)

In [ ]:
labels, df, metrics_list = o.run('data/banking_test.csv', dataset_config, max_items = 100)

## Chain of thought reasoning

Chain of thought requires an explanation for every seed example. As we don't have the explanations or the domain knowledge to construct these explanations, we can use the model to generate the explanations and then use these explanations as an input to the model

## Notes

I am noting a few things that I did not try out, but would have wanted to try to boost completion rate while keeping the accuracy high

1. Wanted to try chain of thought reasoning with a semantic similarity example selector
2. Increase the number of seed examples to be close to one example per class
3. Find the annotator guidelines and pass these as part of the prompt to the model

Trying out all bells and whistles together

In [ ]:
dataset_config = {
    "labels_list": [
        "activate_my_card",
        "age_limit",
        "apple_pay_or_google_pay",
        "atm_support",
        "automatic_top_up",
        "balance_not_updated_after_bank_transfer",
        "balance_not_updated_after_cheque_or_cash_deposit",
        "beneficiary_not_allowed",
        "cancel_transfer",
        "card_about_to_expire",
        "card_acceptance",
        "card_arrival",
        "card_delivery_estimate",
        "card_linking",
        "card_not_working",
        "card_payment_fee_charged",
        "card_payment_not_recognised",
        "card_payment_wrong_exchange_rate",
        "card_swallowed",
        "cash_withdrawal_charge",
        "cash_withdrawal_not_recognised",
        "change_pin",
        "compromised_card",
        "contactless_not_working",
        "country_support",
        "declined_card_payment",
        "declined_cash_withdrawal",
        "declined_transfer",
        "direct_debit_payment_not_recognised",
        "disposable_card_limits",
        "edit_personal_details",
        "exchange_charge",
        "exchange_rate",
        "exchange_via_app",
        "extra_charge_on_statement",
        "failed_transfer",
        "fiat_currency_support",
        "get_disposable_virtual_card",
        "get_physical_card",
        "getting_spare_card",
        "getting_virtual_card",
        "lost_or_stolen_card",
        "lost_or_stolen_phone",
        "order_physical_card",
        "passcode_forgotten",
        "pending_card_payment",
        "pending_cash_withdrawal",
        "pending_top_up",
        "pending_transfer",
        "pin_blocked",
        "receiving_money",
        "Refund_not_showing_up",
        "request_refund",
        "reverted_card_payment?",
        "supported_cards_and_currencies",
        "terminate_account",
        "top_up_by_bank_transfer_charge",
        "top_up_by_card_charge",
        "top_up_by_cash_or_cheque",
        "top_up_failed",
        "top_up_limits",
        "top_up_reverted",
        "topping_up_by_card",
        "transaction_charged_twice",
        "transfer_fee_charged",
        "transfer_into_account",
        "transfer_not_received_by_recipient",
        "transfer_timing",
        "unable_to_verify_identity",
        "verify_my_identity",
        "verify_source_of_funds",
        "verify_top_up",
        "virtual_card_not_working",
        "visa_or_mastercard",
        "why_verify_identity",
        "wrong_amount_of_cash_received",
        "wrong_exchange_rate_for_cash_withdrawal"
    ],
    "dataset_schema": {
        "input_columns": [
            "example"
        ],
        "label_column": "label"
    },
    "seed_examples": 'data/banking_seed.csv'
}

In [ ]:
task_config = {
    "project_name": "BankingClassification",
    "task_type": "classification",
    "prefix_prompt": "You are an expert at understanding twitter complaints.",
    "example_selector": {
        "strategy": "semantic_similarity",
        "num_examples": 4
    },
    "compute_confidence": "True",
    "chain_of_thought": "True"
}

Increase the maximum tokens allowed on the llm in case of chain of thought because some times the number of tokens produced by the llm in the explanation would exceed the default max token limit.

In [ ]:
llm_config = {
    "provider_name": "openai",
    "model_name": "gpt-3.5-turbo",
    "has_logprob": False,
    "model_params": {
        "max_tokens": 1000, # This increases the maximum tokens
    }
}

In [ ]:
o = LabelingAgent(task_config, llm_config)
o.plan('data/banking_test.csv', dataset_config, max_items = 100)

In [ ]:
labels, df, metrics_list = o.run('data/banking_test.csv', dataset_config, max_items = 100)

Increase the maximum tokens allowed on the llm in case of chain of thought because some times the number of tokens produced by the llm in the explanation would exceed the default max token limit.

## Self consistency

This increases the temperature and the randomness while generating explanations allowing the model to explore multiple reasoning paths. At the end, a majority vote is taken among the generations. The llm config below generates 5 reasoning paths and takes the majority vote over these reasoning paths.

In [ ]:
llm_config = {
    "provider_name": "openai",
    "model_name": "gpt-3.5-turbo",
    "has_logprob": False,
    "model_params": {
        "max_tokens": 1000, # This increases the maximum tokens
        "temperature": 0.7,
        "n": 5 # This runs self consistency
    }
}